In [32]:
import requests
from requests.auth import HTTPBasicAuth
from requests.adapters import HTTPAdapter
from urllib3.util.ssl_ import create_urllib3_context

email = 'akash.thakur@voltmoney.in'
api_key = 'tjIk8xDGyQRF1CUIblWI9MWv3nl2Fx4odbYBJwQm'
domain = ''


class TLSAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        context = create_urllib3_context()
        context.set_ciphers('DEFAULT@SECLEVEL=1')  # Adjust cipher settings if needed
        kwargs['ssl_context'] = context
        return super(TLSAdapter, self).init_poolmanager(*args, **kwargs)


url = f'https://{domain}.zendesk.com/api/v2/tickets.json?page[size]=100'



# Create a session
session = requests.Session()
session.mount('https://', TLSAdapter())

def fetch_tickets(url):
    count = 0
    all_tickets = []
    while url :
        print(f'Fetching page {count + 1}')
        if count == 6:
            break
        response = session.get(url, auth=HTTPBasicAuth(f'{email}/token', api_key))
        if response.status_code == 200:
            data = response.json()
            all_tickets.extend(data['tickets'])
            url = data['links'].get('next')  # Get the next page URL
            print
            count += 1
        else:
            print(f'Failed to retrieve tickets. Status code: {response.status_code}')
            print(response.text)
            break
    return all_tickets

# Fetch all tickets
tickets = fetch_tickets(url)
print(tickets)

Fetching page 1
https://volt7307.zendesk.com/api/v2/tickets.json?page%5Bafter%5D=eyJvIjoibmljZV9pZCIsInYiOiJhWlFOQUFBQUFBQUEifQ%3D%3D&page%5Bsize%5D=100
Fetching page 2
https://volt7307.zendesk.com/api/v2/tickets.json?page%5Bafter%5D=eyJvIjoibmljZV9pZCIsInYiOiJhWUFUQUFBQUFBQUEifQ%3D%3D&page%5Bsize%5D=100
Fetching page 3
https://volt7307.zendesk.com/api/v2/tickets.json?page%5Bafter%5D=eyJvIjoibmljZV9pZCIsInYiOiJhV1FXQUFBQUFBQUEifQ%3D%3D&page%5Bsize%5D=100
Fetching page 4
https://volt7307.zendesk.com/api/v2/tickets.json?page%5Bafter%5D=eyJvIjoibmljZV9pZCIsInYiOiJhZU1aQUFBQUFBQUEifQ%3D%3D&page%5Bsize%5D=100
Fetching page 5
https://volt7307.zendesk.com/api/v2/tickets.json?page%5Bafter%5D=eyJvIjoibmljZV9pZCIsInYiOiJhZjRjQUFBQUFBQUEifQ%3D%3D&page%5Bsize%5D=100
Fetching page 6
https://volt7307.zendesk.com/api/v2/tickets.json?page%5Bafter%5D=eyJvIjoibmljZV9pZCIsInYiOiJhUUVoQUFBQUFBQUEifQ%3D%3D&page%5Bsize%5D=100
Fetching page 7
[{'url': 'https://volt7307.zendesk.com/api/v2/tickets/1528.json', 

In [29]:
import pandas as pd

df = pd.json_normalize(data['tickets'])

print(df.columns)

required_columns = {
    'Ticket ID': 'id',
    'Ticket Status': 'status',
    'Subject': 'subject',
    'Type': 'type',
    'Requester': 'requester_id',
    'Assignee': 'assignee_id',
    'Tags': 'tags',
    'Custom Fields': 'custom_fields'
}

df_selected = df[list(required_columns.values())].rename(columns=required_columns)

print(df_selected.columns)

custom_fields_mapping = {
    26280259621649: 'Phone number',
    20458166907025: 'Current Stage',
    # Add other custom field mappings here
}

if 'Custom Fields' in df_selected.columns:
    for field in custom_fields_mapping:
        df_selected[custom_fields_mapping[field]] = df_selected['Custom Fields'].apply(
            lambda x: next((item['value'] for item in x if item['id'] == field), None)
        )

    df_selected.drop(columns=['Custom Fields'], inplace=True)

print(df_selected)

Index(['url', 'id', 'external_id', 'created_at', 'updated_at',
       'generated_timestamp', 'type', 'subject', 'raw_subject', 'description',
       'priority', 'status', 'recipient', 'requester_id', 'submitter_id',
       'assignee_id', 'organization_id', 'group_id', 'collaborator_ids',
       'follower_ids', 'email_cc_ids', 'forum_topic_id', 'problem_id',
       'has_incidents', 'is_public', 'due_at', 'tags', 'custom_fields',
       'satisfaction_rating', 'sharing_agreement_ids', 'custom_status_id',
       'fields', 'followup_ids', 'ticket_form_id', 'brand_id',
       'allow_channelback', 'allow_attachments', 'from_messaging_channel',
       'via.channel', 'via.source.from.address', 'via.source.from.name',
       'via.source.to.name', 'via.source.to.address', 'via.source.rel',
       'via.source.from.ticket_id', 'via.source.from.subject',
       'via.source.from.channel'],
      dtype='object')
Index(['id', 'status', 'subject', 'type', 'requester_id', 'assignee_id',
       'tags', 'c

In [33]:
df_selected.to_csv('df_zendesk_api_selected.csv')

In [12]:
df.to_csv('df_zendesk_api.csv')